In [10]:
from google.colab import files

# Upload the Kaggle dataset zip file
uploaded = files.upload()

# Unzip the dataset
!unzip archive.zip


Saving archive.zip to archive.zip
Archive:  archive.zip
  inflating: annotations/road0.xml   
  inflating: annotations/road1.xml   
  inflating: annotations/road10.xml  
  inflating: annotations/road100.xml  
  inflating: annotations/road101.xml  
  inflating: annotations/road102.xml  
  inflating: annotations/road103.xml  
  inflating: annotations/road104.xml  
  inflating: annotations/road105.xml  
  inflating: annotations/road106.xml  
  inflating: annotations/road107.xml  
  inflating: annotations/road108.xml  
  inflating: annotations/road109.xml  
  inflating: annotations/road11.xml  
  inflating: annotations/road110.xml  
  inflating: annotations/road111.xml  
  inflating: annotations/road112.xml  
  inflating: annotations/road113.xml  
  inflating: annotations/road114.xml  
  inflating: annotations/road115.xml  
  inflating: annotations/road116.xml  
  inflating: annotations/road117.xml  
  inflating: annotations/road118.xml  
  inflating: annotations/road119.xml  
  inflating:

In [11]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import xml.etree.ElementTree as ET

In [38]:
# Path to the directory containing images and annotations
data_path = '/content'

# Loading Images (PNG) and Annotations (XML)
image_folder = os.path.join(data_path, 'images')
annotation_folder = os.path.join(data_path, 'annotations')

# Lists to store features (X) and labels (y)
X = []
y = []

# Iterating over image files
for image_file in os.listdir(image_folder):
    # Load Image
    image_path = os.path.join(image_folder, image_file)
    image = Image.open(image_path)

    # Using the image as a feature
    X.append(image_array)

    # Extracting corresponding label from the XML annotation
    annotation_file = os.path.join(annotation_folder, image_file.replace('.png', '.xml'))
    tree = ET.parse(annotation_file)
    root = tree.getroot()
    class_label = root.find('.//object/name').text
    y.append(class_label)

# Converting lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Creating a KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Training the classifier
knn_classifier.fit(X_train, y_train)

# Making predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Evaluating the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7386363636363636


In [33]:
from sklearn.model_selection import cross_val_score

# Defining a range of hyperparameter values to test
neighbors_range = [3, 5, 7, 9, 11]

# Performing cross-validation for each value of n_neighbors
for n_neighbors in neighbors_range:
    knn_classifier = KNeighborsClassifier(n_neighbors=n_neighbors)
    cv_scores = cross_val_score(knn_classifier, X_train, y_train, cv=5)
    print(f"Mean CV Accuracy (n_neighbors={n_neighbors}): {np.mean(cv_scores)}")

Mean CV Accuracy (n_neighbors=3): 0.6127355623100305
Mean CV Accuracy (n_neighbors=5): 0.7446504559270517
Mean CV Accuracy (n_neighbors=7): 0.7446504559270517
Mean CV Accuracy (n_neighbors=9): 0.7446504559270517
Mean CV Accuracy (n_neighbors=11): 0.7446504559270517


In [34]:
from sklearn.model_selection import GridSearchCV

# Defining the parameter grid
param_grid = {'n_neighbors': [3, 5, 7, 9, 11]}

# Creating a KNN classifier
knn_classifier = KNeighborsClassifier()

# Instantiating the GridSearchCV object
grid_search = GridSearchCV(knn_classifier, param_grid, cv=5, scoring='accuracy')

# Fitting the grid search to the data
grid_search.fit(X_train, y_train)

# Getting the best parameters
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

# Getting the best model
best_knn = grid_search.best_estimator_


Best Parameters: {'n_neighbors': 5}


In [35]:
# Evaluate on the test set
test_accuracy = best_knn.score(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")


Test Accuracy: 0.7386363636363636
